In [ ]:
# 전처리 동안 사용할 라이브러리 종합
import pandas as pd
import numpy as np

import re
import copy
import datetime
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'AppleGothic'
sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
company_info = pd.read_csv('../data/preprocessed_data/samsung.csv',encoding='cp949',index_col=0)
stock_price = pd.read_csv('../data/preprocessed_data/samsung_price.csv',encoding='cp949',index_col=0)

In [ ]:
company_info['보고일자'] = pd.to_datetime(company_info['보고일자'])
stock_price['날짜'] = pd.to_datetime(stock_price['날짜'])

In [ ]:
# 데이터 종합
dataset = pd.merge(stock_price, company_info, how='right', left_on=['날짜'], right_on=['보고일자']).\
    sort_values("보고일자", ascending=False).fillna(method='backfill').reset_index(drop=True)
dataset.drop(columns=["보고일자"], inplace=True)
    
# 날짜데이터를 년/월/일/요일로 구분
dataset["년"] = dataset["날짜"].apply(lambda x: int(datetime.datetime.strftime(x, "%Y-%m-%d").split("-")[0]))
dataset["월"] = dataset["날짜"].apply(lambda x: int(datetime.datetime.strftime(x, "%Y-%m-%d").split("-")[1]))
dataset["일"] = dataset["날짜"].apply(lambda x: int(datetime.datetime.strftime(x, "%Y-%m-%d").split("-")[2]))

temp = dataset["날짜"].dt.weekday
dataset["요일"] = pd.Categorical.from_codes(temp, ["월", "화", "수", "목", "금"], ordered=True)

dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.columns

In [ ]:
dataset.head()

In [ ]:
cols = list(dataset.dtypes[dataset.dtypes == "object"].index)
cols

In [ ]:
for col in cols:
    print("="*50, "\n")
    print(dataset[col].value_counts())
    print("\n카테고리 갯수 :",len(dataset[col].value_counts()), "\n")

In [ ]:
# 직위명 컬럼 정리
temp_pos = dataset["직위명"].value_counts().to_frame().reset_index().rename(columns={"index":"직위명", "직위명":"count"})
temp_pos

In [ ]:
names = list(temp_pos["직위명"].values)
names

In [ ]:
_my_dict = {
    '임원진':['상무','전무','이사','이사대우','사외이사','상무보','대표이사','상임감사','기획지원본부장','임원(실장)',
           '기타비상무이사','본부장','상임이사','이사회의장','임원(제철소부소장)','임원(그룹장)','상무이사','임원(연구소장)','에너지소재본부장',
           '사내이사','위원','분석솔루션센터장','IE소재연구센터장','친환경제품솔루션센터장','ValueCreationCenter장','비상임이사','상무(연구위원)',
           '상ㅁ','임원(국장)','수석연구위원(전무)','의장','신성장동력본부장','홍보실장/SUPEX추구협의회파견/상무','본부임원','전력계통본부장',
           '전문위원','임원(제철소장)','임원(부장)','임원(철강솔루션&cr연구소장)','임원(본부장)','Communication부문장','센터장','FIG리더','부문장',
           '배터리마케팅본부장','ValueCreation&crCenter장','Portfolio부문장','부사장(대행)'],
    
    '대표':['사장','부회장','대표이사사장','회장','대표이사회장','수석부회장','대표이사/CEO','수석부사장','사장(대행)','부사장',
          '사내이사부사장','대표이사부회장','대표이사부사장','수석부사장(대행)','E&P사업대표','대표이사(사장)','대표이사/부회장',
          'E&P사장','기획재무실장','배터리사업대표(사장)','총괄사장'],
    
    '실무진':['담당','실장','연구위원','수석연구위원','소장','PR담당','기술연구소장','연구소장','Compliance담당','Comm.기획담당',
           '기술위원','법무담당','재무실장','IP전략담당','구매담당','사업부장','DT부문장','양극소재실장','내화물생산실장','글로벌로재사업실장',
           'BMR추진담당','세무전략담당','기반기술연구소장'],
    
    '고문':['명예회장','비상근고문','상임고문','상근고문','고문'],
    
    '기타':['자문역','보좌역','직무대행'],
    
    '없음':['-','서울강남구도곡2동'],
}

In [ ]:
_names=[i.strip().replace(' ','') for i in names ]

temp = list()

for k in _my_dict.keys():
    for v in _my_dict[k]:
        temp.append((v,k))

mapper=dict(temp)

In [ ]:
dataset["직위명"] = dataset["직위명"].apply(lambda x: mapper[x.strip().replace(' ',"")])

In [ ]:
dataset["직위명"].unique()

In [ ]:
# EDA분석을 위한 데이터 save
dataset.to_csv("../data/preprocessed_data/forEDA.csv", encoding='cp949')

In [ ]:
# onehot 인코딩 적용할 컬럼 선정 후 처리
cols = list(dataset.dtypes[dataset.dtypes == "object"].index)
cols.append("요일")

temp = pd.DataFrame()
for col in cols:
    temp = pd.concat([temp, pd.get_dummies(dataset[col], prefix=col)], axis=1)

dataset = pd.concat([dataset.drop(columns=cols), temp], axis=1)

# 파생변수 생성
# 1. 당일 거래량/발행주식 총수
dataset["거래량/발행주식 총수"] = dataset["거래량"] / dataset["발행주식 총수"] * 100
# 2. 개인매매/당일 거래량 비율
dataset["당일개인거래비율"] = dataset["개인"]/dataset["거래량"] * 100
# 3. 기관매매/당일 거래량 비율
dataset["당일기관거래비율"] = dataset["기관"]/dataset["거래량"] * 100
# 4. 프로그램/당일 거래량 비율
dataset["당일프로그램거래비율"] = dataset["프로그램"]/dataset["거래량"] * 100 # 모두 % 단위임.

In [ ]:
# 결측치 확인
dataset.isnull().sum().sum()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset[dataset["당일개인거래비율"].isna()]["거래량"]

In [ ]:
# 결측치 제거
dataset.drop(index=dataset[dataset["거래량"] == 0].index, inplace=True)

In [ ]:
dataset.isnull().sum().sum()

In [ ]:
dataset.head()

In [ ]:
# 종속변수와 독립변수 분리
y_data = dataset.loc[:,['날짜', '1일 뒤 종가', '2일 뒤 종가', '3일 뒤 종가', '4일 뒤 종가', '5일 뒤 종가', '6일 뒤 종가','7일 뒤 종가', '8일 뒤 종가', '9일 뒤 종가', '10일 뒤 종가']]
y_data = y_data.drop_duplicates("날짜").set_index("날짜")

X_data = dataset.drop(columns=y_data.columns)

In [ ]:
y_data.head()

In [ ]:
X_data.head()

In [ ]:
print(X_data.shape)
print(y_data.shape)

In [ ]:
temp = X_data.groupby("날짜").mean()
temp.reset_index(inplace=True)

In [ ]:
cnt = X_data.value_counts("날짜").to_frame().reset_index().rename(columns={0:"당일공시갯수"})
cnt

In [ ]:
X_data = pd.merge(temp, cnt, how='inner', on=['날짜']).sort_values("날짜", ascending=False).reset_index(drop=True)

In [ ]:
X_data.shape

In [ ]:
# 독립변수 / 종속변수 저장
X_data.to_csv("../data/preprocessed_data/y_data.csv", encoding='cp949')
y_data.to_csv("../data/preprocessed_data/X_data.csv", encoding='cp949')

In [ ]:
# 데이터셋 저장
dataset.to_csv("../data/preprocessed_data/data_(main).csv", encoding='cp949')

### 이전 기간에 따른 가중치 적용하기

In [ ]:
X_data

In [ ]:
y_data

In [ ]:
temp_y = y_data.reset_index(["날짜"]).copy()
temp_y.rename(columns={"1일 뒤 종가": "1일 뒤",
                     "2일 뒤 종가": "2일 뒤",
                     "3일 뒤 종가": "3일 뒤",
                     "4일 뒤 종가": "4일 뒤",
                     "5일 뒤 종가": "5일 뒤",
                     "6일 뒤 종가": "6일 뒤",
                     "7일 뒤 종가": "7일 뒤",
                     "8일 뒤 종가": "8일 뒤",
                     "9일 뒤 종가": "9일 뒤",
                     "10일 뒤 종가": "10일 뒤"
                     }, inplace=True)

for i in range(1, 11):
    temp_y[f"{i}일 뒤"] = temp_y["날짜"].apply(lambda x: x + datetime.timedelta(days=i))
temp_y.head(3)

In [ ]:
# 가중치 벡터가 담긴 데이터프레임 생성

cols = X_data["날짜"]
dy = 30 # 가중치 적용할 날짜 기준

def weight_arr(n, cols, dy):
    weight = pd.DataFrame(index=temp_y[f"{n}일 뒤"], columns=cols)
    weight = weight.apply(lambda x: cols, axis=1)
    weight.reset_index(inplace=True)
    
    for i in range(X_data.shape[0]):
        weight[i] = (weight[f"{n}일 뒤"]-weight[i]).dt.days

    weight.loc[:,0:] = weight[(weight.loc[:,0:] > 0) & (weight.loc[:,0:] < dy)]
    weight.loc[:,0:] = np.reciprocal(weight.loc[:,0:]).fillna(0)

    weight.set_index(f"{n}일 뒤", inplace=True)
    weight.columns = cols
    return weight

def weight_func(weight_df, temp_df):
    temp_df = temp_df.astype(float).to_numpy()
    weight_df = weight_df.astype(float).to_numpy()

    weight_df = pd.DataFrame(data=weight_df)
    
    df = pd.DataFrame(data=np.dot(weight_df, temp_df))
    return df

In [ ]:
cut_date = cols.iat[-1,] + datetime.timedelta(days=dy)
weight_df = weight_arr(1, cols, dy)
#weight_df = weight_df[weight_df.columns[weight_df.columns > cut_date]]
weight_df

In [ ]:
temp_df = X_data.copy()
#temp_df = temp_df[temp_df["날짜"] > cut_date]
temp_df.set_index("날짜", inplace=True)
temp_df

In [ ]:
weighted_data = weight_func(weight_df, temp_df).values
weighted_data = pd.DataFrame(data=weighted_data, columns=temp_df.columns)
weighted_data.set_index(weight_df.index, inplace=True)

In [ ]:
weighted_data = weighted_data[weighted_data.index > cut_date]
weighted_data.reset_index(drop=True,inplace=True)

In [ ]:
weighted_data["날짜"] = X_data["날짜"]
weighted_data.set_index(["날짜"], inplace=True)

In [ ]:

weighted_data.to_csv("../data/preprocessed_data/weighted_data.csv", encoding='cp949')

In [ ]:
y_data = y_data[y_data.index > cut_date]
y_data

In [ ]:
y_data.to_csv("../data/preprocessed_data/y_data.csv", encoding='cp949')